In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = str(pd.DatetimeIndex(train['일자']).month)
test['월'] = str(pd.DatetimeIndex(test['일자']).month)
train['주'] = str(pd.DatetimeIndex(train['일자']).week)
test['주'] = str(pd.DatetimeIndex(test['일자']).week)
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

<ipython-input-4-048b95631755>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = str(pd.DatetimeIndex(train['일자']).week)
<ipython-input-4-048b95631755>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = str(pd.DatetimeIndex(test['일자']).week)


In [5]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return '3'
    if tomorrow in holidays:
        return '2'
    elif yesterday in holidays:
        return '1'
    else : 
        return '0'

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return str(wom)
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

train['day2']=0
train.loc[train['일']>9, 'day2'] = 1
train.loc[train['일']>19, 'day2'] = 2

test['day2']=0
test.loc[test['일']>9, 'day2'] = 1
test.loc[test['일']>19, 'day2'] = 2

## 공휴일 변수 생성

In [6]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '공휴일전후', '몇주차', '인원변화',
       'day2'],
      dtype='object')

In [7]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일','월','day2','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일','월','day2','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일','월','day2','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일','월','day2','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [8]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일', '월', 'day2', '주', '출근', '휴가비율', '출장비율',
       '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [9]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [10]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [11]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: ['공휴일전후', '몇주차', '요일', '월', '주']
[0 1 3 4 6]


In [12]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [13]:
from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [14]:
lunch_model = AutoMLRegressor(random_state=42, metric=MAE)

lunch_model.fit(lunch_train, y_lunch,
                verbose=3,
                folds=20,
                opt_lvl=3,
                early_stoping=120,
                auto_parameters=False,
                timeout=1100
               )

18:36:25 | > Start Fit Base Model
18:36:44 | ##################################################
18:36:44 | > Start Fit Models 2
18:36:44 | ##################################################
18:36:44 | ##################################################
18:36:45 | > Step 1: calc parameters and pruned score: get test 10 trials
18:37:16 |  One iteration ~ 3.0 sec
18:37:16 |  Possible iters ~ 315.0
18:37:16 | --------------------------------------------------
18:37:16 |   Pruned Threshold Score: 80.2
18:37:16 | ##################################################
18:37:16 | > Step 2: Full opt with Threshold Score Pruner
18:37:16 | ##################################################
18:37:16 | > Start optimization with the parameters:
18:37:16 | CV_Folds = 20
18:37:16 | Score_CV_Folds = 3
18:37:16 | Feature_Selection = False
18:37:16 | Opt_lvl = 3
18:37:16 | Cold_start = 15
18:37:16 | Early_stoping = 120
18:37:16 | Metric = mean_absolute_error
18:37:16 | Direction = minimize
18:37:16 | ########

In [15]:
predicts_Auto_lunch_5 = lunch_model.predict(lunch_test)

In [16]:
dinner_model = AutoMLRegressor(random_state=42, metric=MAE)

dinner_model.fit(dinner_train, y_dinner,         
                 verbose=3,
                 folds=20,
                 opt_lvl=3,
                 early_stoping=120,
                 auto_parameters=False,
                 timeout=1100
                )

18:53:37 | > Start Fit Base Model
18:53:57 | ##################################################
18:53:57 | > Start Fit Models 2
18:53:57 | ##################################################
18:53:57 | ##################################################
18:53:57 | > Step 1: calc parameters and pruned score: get test 10 trials
18:54:29 |  One iteration ~ 3.2 sec
18:54:29 |  Possible iters ~ 298.0
18:54:29 | --------------------------------------------------
18:54:29 |   Pruned Threshold Score: 74.8146
18:54:29 | ##################################################
18:54:29 | > Step 2: Full opt with Threshold Score Pruner
18:54:29 | ##################################################
18:54:29 | > Start optimization with the parameters:
18:54:29 | CV_Folds = 20
18:54:29 | Score_CV_Folds = 3
18:54:29 | Feature_Selection = False
18:54:29 | Opt_lvl = 3
18:54:29 | Cold_start = 15
18:54:29 | Early_stoping = 120
18:54:29 | Metric = mean_absolute_error
18:54:29 | Direction = minimize
18:54:29 | #####

In [17]:
predicts_Auto_dinner_5 = dinner_model.predict(dinner_test)

In [18]:
submission2 = pd.read_csv('../data/sample_submission.csv')

submission2.iloc[:,1] = predicts_Auto_lunch_5
submission2.iloc[:,2] = predicts_Auto_dinner_5
submission2.head()

,일자,중식계,석식계
0,2021-01-27,1014.029644,399.877308
1,2021-01-28,941.116418,417.799554
2,2021-01-29,638.111483,251.144302
3,2021-02-01,1269.110930,508.407070
4,2021-02-02,1064.668107,455.232010


In [20]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission2.to_csv(f'../submission/{today}_autoML_5.csv', index =False)

오늘 날짜 : 20210718
